In [1]:
from requests import HTTPError
import spotipy as spy
import pandas as pd
import csv

In [2]:
with open('secret.txt') as s:
    client_secret = next(s)
s.close()
client_id = '339e9f02e5254f07a945aef3f9471a31'
user = 'Willis Orr'
playlist_uri = 'spotify:playlist:5S8SJdl1BDc0ugpkEvFsIL'

In [3]:
credentials_manager = spy.oauth2.SpotifyClientCredentials(client_id, client_secret)
sp = spy.Spotify(client_credentials_manager=credentials_manager)

In [4]:
pl_t = sp.user_playlist_tracks(user, playlist_uri)
tmp = pl_t['items']
ids = []
while pl_t['next']:
    pl_t = sp.next(pl_t)
    tmp.extend(pl_t['items'])
    for s in tmp:
        ids.append(s["track"]["id"])

KeyboardInterrupt: 

In [ ]:
i = len(ids)
print("Amount of rows:", i)
if len(ids) != len(set(ids)):
    ids = list(set(ids))
    print("Duplicates found, {} rows cleaned.".format(i-len(ids)))
    print("The remaining rows; ", len(ids))
else:
    print("No duplicates")

In [ ]:
f = open('ids.csv', 'w')
writer = csv.writer(f)
writer.writerow(["track id"])
for i in range(len(ids)):
    writer.writerow([ids[i]])
f.close()